<a href="https://colab.research.google.com/github/simonsny/image_background_removal/blob/dev_simon/FlaskApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting Google Colab to your Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing the Flask-ngrok to be able to run the Flask app


In [2]:

!pip install flask-ngrok 


Importing the necessary packages to run the program

In [3]:
import os
import cv2 as cv
import numpy as np
import torch
import torch.nn
from torchvision import transforms, utils
from tqdm import tqdm
from skimage import io, transform, color
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import glob
from __future__ import print_function, division
import random
import math
import matplotlib.pyplot as plt

Getting the inference from U2Net to get the alpha matte which we will use as a binary mask to get the trimap.

In [4]:
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob
%cd /content/drive/MyDrive/BeCode/Faktion/U-2-Net/
from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET # full size version 173.6 MB
from model import U2NETP # small version u2net 4.7 MB

# normalize the predicted SOD probability map
def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def save_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split(os.sep)[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    imo.save(d_dir+imidx+'.png')

def u2net(filename):
    
    %cd /content/drive/MyDrive/BeCode/Faktion/U-2-Net/

    # --------- 1. get image path and name ---------
    model_name='u2net'#u2netp

    """
    image_dir = os.path.join(os.getcwd(), 'test_data', 'test_images')
    prediction_dir = os.path.join(os.getcwd(), 'test_data', model_name + '_results' + os.sep)
    model_dir = os.path.join(os.getcwd(), 'saved_models', model_name, model_name + '.pth')
    """
    image_dir = filename
    #image_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/uploads/'
    prediction_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/binary_mask/'
    model_dir =  '/content/drive/MyDrive/BeCode/Faktion/U-2-Net/saved_models/u2net/u2net.pth'


    img_name_list = [filename.split("/")[-1]]
    print(img_name_list)


    %cd '/content/drive/MyDrive/BeCode/Faktion/Flask/static/uploads/' 
    # --------- 2. dataloader ---------
    #1. dataloader
    test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                        lbl_name_list = [],
                                        transform=transforms.Compose([RescaleT(320),
                                                                      ToTensorLab(flag=0)])
                                        )
    test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                        batch_size=1,
                                        shuffle=False,
                                        num_workers=1)

    # --------- 3. model define ---------
    if(model_name=='u2net'):
        print("...load U2NET---173.6 MB")
        net = U2NET(3,1)
    elif(model_name=='u2netp'):
        print("...load U2NEP---4.7 MB")
        net = U2NETP(3,1)

    if torch.cuda.is_available():
        net.load_state_dict(torch.load(model_dir))
        net.cuda()
    else:
        net.load_state_dict(torch.load(model_dir, map_location='cpu'))
    net.eval()

    # --------- 4. inference for each image ---------
    for i_test, data_test in enumerate(test_salobj_dataloader):

        print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

        inputs_test = data_test['image']
        inputs_test = inputs_test.type(torch.FloatTensor)

        if torch.cuda.is_available():
            inputs_test = Variable(inputs_test.cuda())
        else:
            inputs_test = Variable(inputs_test)

        d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

        # normalization
        pred = d1[:,0,:,:]
        pred = normPRED(pred)

        # save results to test_results folder
        if not os.path.exists(prediction_dir):
            os.makedirs(prediction_dir, exist_ok=True)
        save_output(img_name_list[i_test],pred,prediction_dir)

        del d1,d2,d3,d4,d5,d6,d7

    print("Successfully created the binary mask")
    return filename.split(".")[0] + ".png"


/content/drive/.shortcut-targets-by-id/1q6pqWmSu1bY-mc2AFRwGmQIGMV0sa6EY/BeCode/Faktion/U-2-Net


Generate the trimap using the U2net mask

In [49]:
import cv2
"""
def generate_trimap(mask_path,eroision_iter=3,dilate_iter=6):
    mask =  mask_path
    mask = cv2.imread(mask,0)
    mask[mask==1] = 255
    d_kernel = np.ones((3,3))
    erode  = cv2.erode(mask,d_kernel,iterations=eroision_iter)
    dilate = cv2.dilate(mask,d_kernel,iterations=dilate_iter)
    unknown1 = cv2.bitwise_xor(erode,mask)
    unknown2 = cv2.bitwise_xor(dilate,mask)
    unknowns = cv2.add(unknown1,unknown2)
    unknowns[unknowns==255]=127
    trimap = cv2.add(mask,unknowns)
    cv2.imwrite(f"/content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/{mask_path.split('/')[-1]}",trimap)
    return mask_path.split('/')[-1]
    """

# Trying out Simon's Trimap code again :)

def generate_trimap(mask_path,eroision_iter=2,dilate_iter=4):
    mask =  mask_path
    mask = cv2.imread(mask,0)
    mask[mask >= 20] = 255
    mask[mask < 20] = 0
    d_kernel = np.ones((3,3))
    erode  = cv2.erode(mask,d_kernel,iterations=eroision_iter)
    dilate = cv2.dilate(mask,d_kernel,iterations=dilate_iter)

    gray = (dilate-erode)/2
    trimap = erode + gray
    cv2.imwrite(f"/content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/{mask_path.split('/')[-1]}",trimap)
    return mask_path.split('/')[-1]
    

Generate the Alpha Matte using DIM

In [50]:
from torchvision import transforms
%cd /content/drive/MyDrive/Faktion/model_code

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # sets device for model and PyTorch tensors

def alpha_matte(file):
  %cd /content/drive/MyDrive/BeCode/Faktion/model_code/
  IMG_FOLDER = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/uploads/'
  TRIMAP_FOLDER = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/'
  matte_folder = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/matte/'
  print("Alpha Matte Function")
  checkpoint = '/content/drive/MyDrive/BeCode/Faktion/BEST_checkpoint.tar'
  checkpoint = torch.load(checkpoint)
  model = checkpoint['model'].module
  model = model.to(device)
  model.eval()
  print('DIM Model Eval')

  filename = os.path.join(IMG_FOLDER, file)
  img = cv.imread(filename)
  print(img.shape)
  h, w = img.shape[:2]


  x = torch.zeros((1, 4, h, w), dtype=torch.float)
  image = img[..., ::-1]  # RGB
  image = transforms.ToPILImage()(image)

  transformer = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  image = transformer(image)
  x[0:, 0:3, :, :] = image

  filename = os.path.join(TRIMAP_FOLDER, file.replace('.jpg', '.png'))
  print(f'Reading trimap: {filename}...')
  trimap = cv.imread(filename, 0)
  
  x[0:, 3, :, :] = torch.from_numpy(trimap.copy() / 255.)
  
  # Move to GPU, if available
  x = x.type(torch.FloatTensor).to(device)

  with torch.no_grad():
      pred = model(x)

  pred = pred.cpu().numpy()
  pred = pred.reshape((h, w))

  pred[trimap == 0] = 0.0
  pred[trimap == 255] = 1.0

  out = (pred.copy() * 255).astype(np.uint8)

  #filename = os.path.join(OUTPUT_FOLDERS[i], file)
  #cv.imread(filename)
  cv.imwrite(matte_folder+file.replace('.jpg', '.png'), out)
  print(f'Created an alpha matte for  {file}.'.format(filename))
  return matte_folder + file.replace('.jpg', '.png')




/content/drive/.shortcut-targets-by-id/1pbfU2Nkn_OKUrAgN_cnPqhS91KoSQBr8/Faktion/model_code


Getting the foreground image using the original image and the alpha matte.


In [52]:
def foreground_image(image, matte, filename):
  print(f'Image name: {filename}')
  # obtain predicted foreground
  output_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/output/'
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  foreground_name = filename.split('.')[0] + '.png'
  Image.fromarray(((foreground).astype('uint8')), mode='L').save(os.path.join(outpur_dir, foreground_name))
  print(f"Successfully saved the foreground image {foreground_name}")
  return foreground_name


def foreground_img(image, alpha, filename):
  # Read the images
  img = cv2.imread(image)
  matte = cv2.imread(alpha)

  # Convert uint8 to float
  img = img.astype(float)

  # Normalize the alpha matte mask to keep intensity between 0 and 1
  matte = matte.astype(float)/255

  # Multiply the foreground with the alpha matte
  foreground = cv2.multiply(matte, img)

  # Save image
  cv2.imwrite(output_dir+filename.replace('.jpg', '.png'),  foreground)
  print(f"Successfully saved the foreground image for {filename}.")
  return filename

Flask Application starts here.

In [58]:
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, flash, render_template, request, redirect, url_for, send_file
import time
from werkzeug.utils import secure_filename
from google.colab.patches import cv2_imshow

"""
:attrib upload_dir contains the upload path of the uploaded image
:attrib output_dir contains the output/result image 
:attrib matte_dir contains the alpha matte of the image
:allowed_extensions contains the list of allowed image file extensions
"""
upload_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/uploads/'
output_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/output/'
matte_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/matte/'
trimap_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/'
binary_dir = '/content/drive/MyDrive/BeCode/Faktion/Flask/static/binary_mask/'
allowed_extensions = {"png", "jpg", "jpeg", "gif"}

app = Flask(__name__, template_folder='/content/drive/MyDrive/BeCode/Faktion/Flask/templates/', static_folder='/content/drive/MyDrive/BeCode/Faktion/Flask/static')
run_with_ngrok(app)
app.secret_key = "secret key"
app.config["UPLOAD_FOLDER"] = upload_dir

def allowed_file(filename):
    """
    Function that checks the file extensions is included on the allowed list.
    """
    return "." in filename and filename.rsplit(".", 1)[1].lower() in allowed_extensions

@app.route("/")
def home():
    """
    Function that returns the home page.
    """
    return render_template("home.html")

@app.route("/start", methods=["GET", "POST"])
def start():
    """
    Function that has both GET and POST method.
    This is the function where it will ask the user input and 
    then return the input with the edited version of the input
    """
    if request.method == "GET":
        return render_template("start.html")

    if request.method == "POST":
        if "image" not in request.files and "video" not in request.files:
            return redirect(request.url)
        """
        :attrib image will contain the image user input 
        :attrib video will contain the video user input
        :attrib start will contain the start time on when the program started
        :attrib end will contain the end time on when the program ended
        """
        image = request.files.get("image")
        print(f"Image Input: {image}")
        video = request.files.get("video")
        print(f"Video Input: {video}")
        start = time.time()

        if image == "" and video == "":
            return redirect(request.url)

        elif image:
            print("Elif image file")
            if allowed_file(image.filename):
                image_upload = secure_filename(image.filename)
                image.save(os.path.join(app.config["UPLOAD_FOLDER"], image_upload))
                #calling the function to create the alpha matte
                #matte_name = image_ckpt(image_upload)

                #get the binary mask using U2Net
                binary_mask = u2net(image_upload)


                
                #generate trimap using binary mask
                trimap = generate_trimap(binary_dir + binary_mask)
                #trimap = gen_trimap(binary_dir + binary_mask)
                #get the alpha matte which needs image and trimap
                matte_name = alpha_matte(image.filename)

                #calling the function to create the foreground image
                #img_path = Image.open(os.path.join(upload_dir, image.filename))
                #matte_path = Image.open(os.path.join(matte_dir, matte_name))

                image_path = upload_dir + image.filename
                print(f'Image Path: {image_path}')
                print(f'Binary Mask Path: {binary_dir + binary_mask}')
                #print(f'Trimap Path: {trimap_dir +trimap}')
                print(f'Alpha Matte Path: {matte_name}')


                edited_image = foreground_img(image_path, matte_name, image.filename)
                print(f'Edited Image: {edited_image}')
                foreground = Image.open(os.path.join(output_dir, edited_image))


                end = time.time()
                print(f"Program runs for {end - start} seconds.")
                return render_template("upload_image.html", filename=image_upload)
            else:
                flash("Chosen file is not supported! Please upload an image file.")
                flash("Allowed image types are -> png, jpg, jpeg, gif")
                return redirect(request.url)

        elif video:
            print("Elif video file")
            end = time.time()
            print(f"Program runs for {end - start} seconds.")
            return render_template("home.html")

        else:
            return render_template("start.html")

@app.route("/display/<filename>")
def display_image(filename):
    """
    Function that displays the uploaded image.
    """
    print(f"Display image : {filename}")
    return redirect(url_for("static", filename="uploads/" + filename), code=301)
    
@app.route("/edited/<filename>")
def edited_image(filename):
    """
    Function that displayes the edited image/result image.
    """
    file = filename.replace('.jpg', '.png')
    print(f"Edited image : {file}")
    return redirect(url_for("static", filename="output/" + file), code=301)

@app.route("/save_file/<filename>")
def save_file(filename):
    """
    Function that that allows the user to save/download the image.
    """
    path = f"/content/drive/MyDrive/BeCode/Faktion/Flask/static/output/{filename.replace('.jpg', '.png')}"
    print(f"Send File Path: {path}")
    return send_file(path, as_attachment=True)

@app.route("/upload_image")
def upload_image():
    """
    Function that returns the upload_image page.
    """
    return render_template("upload_image.html")

@app.route("/howtouse")
def howtouse():
    """
    Function that returns the how to use page.
    """
    return render_template("howtouse.html")

@app.route("/about")
def about():
    """
    Function that returns the about page.
    """
    return render_template("about.html")

@app.route("/live_feed")
def live_feed():
    """
    Function that returns the live feed page.
    """
    return render_template("live_feed.html")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://09e4c0f0141f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2021 16:40:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:07] "GET /static/assets/logo4.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:07] "GET /static/assets/cover.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:07] "GET /static/assets/remove.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:07] "GET /static/assets/favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:08] "GET /start HTTP/1.1" 200 -


Image Input: <FileStorage: 'ILSVRC2012_test_00000181.jpg' ('image/jpeg')>
Video Input: None
Elif image file
/content/drive/.shortcut-targets-by-id/1q6pqWmSu1bY-mc2AFRwGmQIGMV0sa6EY/BeCode/Faktion/U-2-Net
['ILSVRC2012_test_00000181.jpg']
/content/drive/.shortcut-targets-by-id/1q6pqWmSu1bY-mc2AFRwGmQIGMV0sa6EY/BeCode/Faktion/Flask/static/uploads
...load U2NET---173.6 MB
inferencing: ILSVRC2012_test_00000181.jpg
Successfully created the binary mask
/content/drive/.shortcut-targets-by-id/1q6pqWmSu1bY-mc2AFRwGmQIGMV0sa6EY/BeCode/Faktion/model_code
Alpha Matte Function


...

DIM Model Eval
(397, 400, 3)
Reading trimap: /content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/ILSVRC2012_test_00000181.png...
Created an alpha matte for  ILSVRC2012_test_00000181.jpg.
Image Path: /content/drive/MyDrive/BeCode/Faktion/Flask/static/uploads/ILSVRC2012_test_00000181.jpg
Binary Mask Path: /content/drive/MyDrive/BeCode/Faktion/Flask/static/binary_mask/ILSVRC2012_test_00000181.png
Alpha Matte Path: /content/drive/MyDrive/BeCode/Faktion/Flask/static/matte/ILSVRC2012_test_00000181.png
Successfully saved the foreground image for ILSVRC2012_test_00000181.jpg.
Edited Image: ILSVRC2012_test_00000181.jpg
Program runs for 1.5996310710906982 seconds.


127.0.0.1 - - [09/Jun/2021 16:40:15] "GET /display/ILSVRC2012_test_00000181.jpg HTTP/1.1" 301 -


Display image : ILSVRC2012_test_00000181.jpg


127.0.0.1 - - [09/Jun/2021 16:40:15] "GET /static/uploads/ILSVRC2012_test_00000181.jpg HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 16:40:15] "GET /edited/ILSVRC2012_test_00000181.jpg HTTP/1.1" 301 -
127.0.0.1 - - [09/Jun/2021 16:40:15] "GET /static/output/ILSVRC2012_test_00000181.png HTTP/1.1" 200 -


Edited image : ILSVRC2012_test_00000181.png
